# SETUP

In [ ]:
# Matt
import sys
import os
sys.path.append(os.path.abspath('../combots_venv/lib/python3.12/site-packages'))
%pip install cltl.combot --break-system-packages

from dotenv import load_dotenv
# Get the current working directory
current_dir = os.getcwd()
# Construct the relative path to the .env file
env_path = os.path.join(current_dir, '../.env')
# Load the .env file
load_dotenv(env_path)


# OpenAI API Key
api_key = os.getenv("OPENAI_API_KEY")

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Colab
!pip install --upgrade ai2thor --quiet
!pip install ai2thor-colab prior --upgrade &> /dev/null
!pip install python-dotenv
%pip install cltl.combot --break-system-packages
import os
os.system('apt-get install xvfb')
import ai2thor_colab
ai2thor_colab.start_xserver()

from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# OpenAI API Key
api_key = os.getenv("OPENAI_API_KEY")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.4/347.4 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of marshmallow-dataclass to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.0 MB/s eta 0:00:00
  Created wheel for emissor: filename=emissor-0.0.dev6-py3-none-any.whl size=28072 sha

  if event.key is 'enter':



In [2]:
import prior

dataset = prior.load_dataset("procthor-10k")
dataset

[AI2-THOR WARNING] There has been an update to ProcTHOR-10K that must be used with AI2-THOR version 5.0+. To use the new version of ProcTHOR-10K, please update AI2-THOR to version 5.0+ by running:
    pip install --upgrade ai2thor
Alternatively, to downgrade to the old version of ProcTHOR-10K, run:
   prior.load_dataset("procthor-10k", revision="ab3cacd0fc17754d4c080a3fd50b18395fae8647")


Loading test: 100%|██████████| 1000/1000 [00:00<00:00, 2950.32it/s]


DatasetDict(
    train=Dataset(
    dataset=procthor-dataset,
    size=10000,
    split=train
),
    val=Dataset(
    dataset=procthor-dataset,
    size=1000,
    split=val
),
    test=Dataset(
    dataset=procthor-dataset,
    size=1000,
    split=test
)
)

In [3]:
house = dataset["train"][11] # CHOOSE HOUSE

In [4]:
from ai2thor.controller import Controller
from PIL import Image

In [5]:
controller = Controller(scene=house, visibilityDistance=10, width=750, height=750)

thor-Linux64-f0825767cd50d69f666c7f282e54abfe58f1e917.zip: [ 100%  31.7 MiB/s]  of 769.MB


In [ ]:
event = controller.step(action="GetReachablePositions")
reachable_positions = event.metadata["actionReturn"]

In [ ]:
from openaiapi import analyze_prompt, analyze_image
from utils import numpy_to_base64

frame = controller.last_event.frame
base64_string = numpy_to_base64(frame)

#FUNCTIONS

In [12]:
import math

def teleport_in_front_of_object(controller, object_position, reachable_positions, distance=1.0):
  """Teleports the agent to the closest reachable position in front of an object.

  Args:
    controller: The AI2Thor controller.
    object_position: The position of the target object.
    reachable_positions: A list of reachable positions in the scene.
    distance: The desired distance in front of the object.

  Returns:
    The event after teleporting.
  """

  # Calculate the target position in front of the object
  target_position = {
      "x": object_position["x"] - distance,
      "y": object_position["y"],
      "z": object_position["z"]
  }

  # Find the closest reachable position
  closest_position = None
  min_distance = float('inf')

  for position in reachable_positions:
    dist = math.sqrt((position["x"] - target_position["x"])**2 +
                     (position["z"] - target_position["z"])**2)
    if dist < min_distance:
      min_distance = dist
      closest_position = position

# Calculate rotation towards the object
  dx = object_position["x"] - closest_position["x"]
  dz = object_position["z"] - closest_position["z"]
  rotation = math.degrees(math.atan2(dx, dz))

  # Teleport and rotate
  event = controller.step(action="Teleport", position=closest_position, rotation=rotation)

  return event  # Return the event after adjusting view angle

In [13]:
def get_object_positions(controller, matched_object):
  """
  Finds the positions of all visible objects of a specific type.

  Args:
    controller: The AI2Thor controller.
    matched_object: The type of object to find (e.g., "Painting", "Chair", "Table").

  Returns:
    A list of positions for the specified object type.
  """
  visible_objects = [obj for obj in controller.last_event.metadata["objects"] if obj["visible"]]
  objects_of_interest = [obj for obj in visible_objects if obj["objectType"] == matched_object]
  object_positions = []
  for obj in objects_of_interest:
      #print(obj["name"], obj["position"])
      object_positions.append(obj["position"])
  return object_positions

In [35]:
import re

def interactive_object_match(api_key: str, human_object_description: str, unique_object_list: list, HUMAN: str, AGENT: str, leolaniClient):
    """
    Interactively matches a human description of an object to one from a given list using an LLM.
    The function continues to refine guesses based on user confirmation or denial.

    Args:
        api_key (str): The API key for accessing the LLM.
        human_object_description (str): A description of the object to match.
        unique_object_list (list): The list of unique objects to match against.

    Returns:
        str: The confirmed object from the user.
        list: The matched object(s) from the list based on the LLM's response.
    """
    def ask_llm(description: str, objects: list) -> str:
        """Helper function to query the LLM for matching the description."""
        object_list_str = ", ".join(objects)
        prompt = (
            f"Imagine you are tasked with identifying an object from a given list based on its description. "
            f"The list of objects is: {object_list_str}. "
            f"Your task is to match the following description to one or more objects from the list: \n"
            f"'{description}'\n\n"
            "If you have a single best guess, respond with: 'To be sure, would you describe your object as {object}?'\n"
            "If you are unsure and need clarification between a few options, respond with: "
            "'To be sure, would you describe your object as {object1} or {object2}?'"
            "Only use objects from the list."
        )
        # Make sure the response is extracted correctly from the LLM
        llm_response = analyze_prompt(api_key=api_key, prompt=prompt)

        if isinstance(llm_response, tuple):
            llm_response = llm_response[0]

        if isinstance(llm_response, list) and llm_response:  # Check if it's a non-empty list
            llm_response = llm_response[0]  # Access the first element if it's a list

        if isinstance(llm_response, dict) and "choices" in llm_response and llm_response["choices"]:
            return llm_response["choices"][0]["message"]["content"]
        return llm_response  # Return raw response if format is unexpected

    current_description = human_object_description

    while True:
        # Query the LLM for a guess
        response = ask_llm(current_description, unique_object_list)
        leolaniClient._add_utterance(AGENT, response)
        print(f"{AGENT}>{response}")

        # Extract the matched object(s) from the response
        matched_objects = re.findall(r"\b(" + "|".join(map(re.escape, unique_object_list)) + r")\b", response)

        # Ask the user for confirmation or denial
        confirmation_prompt = "Is this correct? (yes/no): "
        leolaniClient._add_utterance(AGENT, confirmation_prompt)
        print(f"{AGENT}>{confirmation_prompt}")
        user_input = input().strip().lower()
        leolaniClient._add_utterance(HUMAN, user_input)
        print(f"{HUMAN}>{user_input}")

        if user_input == "yes":
            success_message = "Great! Object successfully matched."
            leolaniClient._add_utterance(AGENT, success_message)
            print(f"{AGENT}>{success_message}")
            return matched_objects[0] if len(matched_objects) == 1 else matched_objects
        elif user_input == "no":
            refine_message = "Let's refine the search. Can you provide more details or clarify the description?"
            leolaniClient._add_utterance(AGENT, refine_message)
            print(f"{AGENT}>{refine_message}")
            clarifying_question = input().strip()
            leolaniClient._add_utterance(HUMAN, clarifying_question)
            print(f"{HUMAN}>{clarifying_question}")
            current_description += " " + clarifying_question
        else:
            error_message = "Please respond with 'yes' or 'no'."
            leolaniClient._add_utterance(AGENT, error_message)
            print(f"{AGENT}>{error_message}")


In [28]:
unique_object_list = [
    "AlarmClock", "AluminumFoil", "Apple", "ArmChair", "BaseballBat", "BasketBall",
    "Bathtub", "BathtubBasin", "Bed", "Blinds", "Book", "Boots", "Bottle", "Bowl",
    "Box", "Bread", "ButterKnife", "Cabinet", "Candle", "CD", "CellPhone", "Chair",
    "Cloth", "CoffeeMachine", "CoffeeTable", "CounterTop", "CreditCard", "Cup",
    "Curtains", "Desk", "DeskLamp", "Desktop", "DiningTable", "DishSponge", "DogBed",
    "Drawer", "Dresser", "Dumbbell", "Egg", "Faucet", "Floor", "FloorLamp",
    "Footstool", "Fork", "Fridge", "GarbageBag", "GarbageCan", "HandTowel",
    "HandTowelHolder", "HousePlant", "Kettle", "KeyChain", "Knife", "Ladle", "Laptop",
    "LaundryHamper", "Lettuce", "LightSwitch", "Microwave", "Mirror", "Mug",
    "Newspaper", "Ottoman", "Painting", "Pan", "PaperTowelRoll", "Pen", "Pencil",
    "PepperShaker", "Pillow", "Plate", "Plunger", "Poster", "Pot", "Potato",
    "RemoteControl", "RoomDecor", "Safe", "SaltShaker", "ScrubBrush", "Shelf",
    "ShelvingUnit", "ShowerCurtain", "ShowerDoor", "ShowerGlass", "ShowerHead",
    "SideTable", "Sink", "SinkBasin", "SoapBar", "SoapBottle", "Sofa", "Spatula",
    "Spoon", "SprayBottle", "Statue", "Stool", "StoveBurner", "StoveKnob",
    "TableTopDecor", "TargetCircle", "TeddyBear", "Television", "TennisRacket",
    "TissueBox", "Toaster", "Toilet", "ToiletPaper", "ToiletPaperHanger", "Tomato",
    "Towel", "TowelHolder", "TVStand", "VacuumCleaner", "Vase", "Watch",
    "WateringCan", "Window", "WineBottle"
]

# Test Interaction

In [14]:
# adding to the system path
import sys
sys.path.insert(0, os.path.abspath('../emissor_chat'))

from leolani_client import LeolaniChatClient, Action
emissor_path = "./emissor"
AGENT="Ai2Thor"
HUMAN = "Human"
leolaniClient = LeolaniChatClient(emissor_path=emissor_path, agent=AGENT, human=HUMAN)

In [17]:
utterance = f"Hi {HUMAN}. What do you see in the room?"
print(AGENT+">"+utterance)
leolaniClient._add_utterance(AGENT, utterance)

Ai2Thor>Hi Human. What do you see in the room?


In [ ]:
# grab img of an item to look for

#example: look for a tv in a room
#event = controller.step(action="Teleport", position={'x': 2.75, 'y': 0.9009997844696045, 'z': 1.0}  with rotation 209)
#Image.fromarray(event.frame)

In [18]:
human_room_description = "there is a table. 5 chairs. wthere is a window. its probably a living room."

In [19]:
print(HUMAN+">"+human_room_description)
leolaniClient._add_utterance(HUMAN, human_room_description)

Human>there is a table. 5 chairs. wthere is a window. its probably a living room.


In [20]:
# claryfying questions
claryfying_questions_response = analyze_prompt(api_key=api_key, model = "gpt-4o-mini", prompt=f"Imagine you are a robot who needs to be on a exact location as the point of view that the human has. After a while, the human can no longer see this image. The human will most likely describe a room from memory. The human will most likely describe a few objects and maybe some other attributes, like colours of objects. Your task is to ask claryfing questions about the room and objects so that you (the robot) has the highest chance of finding where the human was standing. Remember, ask the questions as if you were directly talking to the human. Try not to ask for too much details and dont ask for too much; remember, the human has to describe the image from memory, so only ask what you deem most important. \n Human description: {human_room_description}")


In [21]:
utterance = claryfying_questions_response[0]["choices"][0]["message"]["content"]
print(AGENT+">"+utterance)
leolaniClient._add_utterance(AGENT, utterance)

Ai2Thor>Thank you for the description! To help me locate where you were standing, could you tell me the color of the table and the chairs? Also, what size is the window, and where is it positioned in the room?


In [22]:
human_room_description_clarified = "The table is blue, chairs are all black. The window is on the left wall in the same corner as th balcony doors."
print(HUMAN+">"+human_room_description_clarified)
leolaniClient._add_utterance(HUMAN, human_room_description)

Human>The table is blue, chairs are all black. The window is on the left wall in the same corner as th balcony doors.


In [23]:
utterance = "Describe the object I should look for."
print(AGENT+">"+utterance)
leolaniClient._add_utterance(AGENT, utterance)

Ai2Thor>Describe the object I should look for.


In [24]:
human_obj_description = "It's a dark painting with trees a moon. some clouds, a river."

In [25]:
print(HUMAN+">"+human_obj_description)
leolaniClient._add_utterance(HUMAN, human_room_description)

Human>It's a dark painting with trees a moon. some clouds, a river.


In [36]:
# this is where the interactive object match comes :
# based on "dark painting with trees ...", did you mean "Painting"?

human_object_description = human_obj_description
matched_object = interactive_object_match(api_key=api_key, human_object_description=human_object_description, unique_object_list=unique_object_list, HUMAN=HUMAN, AGENT=AGENT, leolaniClient=leolaniClient)
print(f"Final matched object(s): {matched_object}")


Ai2Thor>To be sure, would you describe your object as Painting?
Ai2Thor>Is this correct? (yes/no): 
yes
Human>yes
Ai2Thor>Great! Object successfully matched.
Final matched object(s): Painting


In [ ]:
# Teleport somewhere random
import random

position = random.choice(reachable_positions)
rotation = random.choice(range(360))
print("Teleporting the agent to", position, " with rotation", rotation)

event = controller.step(action="Teleport", position=position, rotation=rotation)

#Image.fromarray(event.frame) # image for clearity

Teleporting the agent to {'x': 0.75, 'y': 0.9009997844696045, 'z': 4.25}  with rotation 264


In [ ]:
#location classificaiton:

# chatgpt, do you think you are in the correct room based on human_room_description + human_room description_clarified? (maybe rotate 360 degrees, but also have to analyze 4 images then)

room_classifcation = analyze_image(base64_string, api_key=api_key, prompt= f"Imagine you are a robot looking for a certain room. Describe the room shortly. Do you think you are in the correct room based on the following description? Description: {human_room_description}, even further description: {human_room_description_clarified}, speak to me as if you have the robots point of view. Be consise in your answer.")
utterance = room_classifcation[0]["choices"][0]["message"]["content"]
print(AGENT+">"+utterance)
leolaniClient._add_utterance(AGENT, utterance)


Ai2Thor>Based on the provided image, I observe a painting on the wall, but no table, chairs, or windows visible. This does not match the description of a living room with a blue table, black chairs, and a window next to balcony doors. I do not think I am in the correct room.


In [ ]:
# Decide what to do:  teleport to another room, teleport to object instance (if there is one) or lookleft/lookright etc

In [ ]:
#Metadata

# i see x instances of matched_object
# teleport to matched_object[0]
# use chatgpt to describe image
# do so until image is found or no instances left

#matched_object = "Painting" # placeholder

object_positions = get_object_positions(controller, matched_object)

for i in range(len(object_positions)):
  position = object_positions[i]
  event = teleport_in_front_of_object(controller, position, reachable_positions)

  description = analyze_image(base64_string, api_key=api_key, prompt= f"describe {matched_object} in great detail")
  utterance = description[0]["choices"][0]["message"]["content"]

  print(AGENT+">"+utterance+"Was this the item you were looking for?")
  leolaniClient._add_utterance(AGENT, utterance+"Was this the item you were looking for?")
  if input("Type yes if so...") == "yes":
    print(HUMAN+">"+"yes")
    leolaniClient._add_utterance(HUMAN, "yes")
    break
  else:
      print(HUMAN+">"+"no")
      leolaniClient._add_utterance(HUMAN, "no")
      continue

print("Teleport to new room if item isn't found")


Ai2Thor>The painting depicts a serene nocturnal landscape. At the center, a luminous full moon is prominent, partially obscured by dramatic, swirling clouds that add texture to the night sky. The moonlight casts an ethereal glow over the scene, highlighting the silhouettes of dense trees on either side, which frame the composition.

In the middle ground, a calm body of water, possibly a pond or small lake, reflects the moonlight, contributing to the tranquil atmosphere. The reflection creates a mirror-like effect thatWas this the item you were looking for?
Type yes if so...no
Human>no
Teleport to new room


In [ ]:
# if no more instances, teleport to new room

In [ ]:
##### After completion, we save the scenario in the defined emissor folder.
leolaniClient._save_scenario()

# EXAMPLE IMAGES

RANDOM TELEPORT

In [ ]:
import random

position = random.choice(reachable_positions)
rotation = random.choice(range(360))
print("Teleporting the agent to", position, " with rotation", rotation)

event = controller.step(action="Teleport", position=position, rotation=rotation)
Image.fromarray(event.frame)

IMAGE WITH 2 PAINTING

In [ ]:
event = controller.step(action="Teleport", position={'x': 2.5, 'y': 0.9009997844696045, 'z': 4.5}, rotation=80)
Image.fromarray(event.frame)

#TESTING

ROTATING AND REMEMBERING OBJECT LOCATIONS

In [ ]:
visible_objects = [obj for obj in controller.last_event.metadata["objects"] if obj["visible"]]
paintings = [obj for obj in visible_objects if obj["objectType"] == "Painting"]
all_painting_positions = []  # To store all painting positions

for _ in range(3):  # Rotate three times
    # Get visible paintings and their positions
    current_painting_positions = []
    for painting in paintings:
        print(painting["name"], painting["position"])
        current_painting_positions.append(painting["position"])

    # Add current painting positions to the overall list
    all_painting_positions.extend(current_painting_positions)

    # Rotate the agent
    controller.step("RotateRight")

    # Update visible objects and paintings for the next iteration
    visible_objects = [obj for obj in controller.last_event.metadata["objects"] if obj["visible"]]
    paintings = [obj for obj in visible_objects if obj["objectType"] == "Painting"]

print("All painting positions in the room:", all_painting_positions)

LOOK UP / DOWN

In [ ]:
#controller.step("LookDown")
#Image.fromarray(controller.last_event.frame)

# after we take look up or down we should return to the original state, THIS IS IMPORTANT OTHERWISE IT WILL MESS WITH TELEPORTING